<a href="https://colab.research.google.com/github/sayakpaul/EvoNorms-in-TensorFlow-2/blob/master/Mini_Inception_EvoNorm_Sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Which GPU?
!nvidia-smi

In [2]:
# TensorFlow imports
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [0]:
# Other imports
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import numpy as np

In [0]:
# Set the random seeds
tf.random.set_seed(666)
np.random.seed(666)

In [0]:
# Set up wandb for easy experiment tracking
!pip install wandb -q
import wandb
wandb.login()

In [6]:
# Load and preprocess CIFAR10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train / 255.
X_test = X_test / 255.
print(X_train.shape, X_test.shape)

170500096/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)


## `EvoNorm2dS0`

In [0]:
# Reference
# https://github.com/lonePatient/EvoNorms_PyTorch/blob/master/models/normalization.py

def group_std(x, groups=32, eps=1e-5):
	N, H, W, C = tf.shape(x)
	x = tf.reshape(x, [N, H, W, groups, C // groups])
	_, var = tf.nn.moments(x, [1, 2, 4], keepdims=True)
	std = tf.sqrt(var + eps)
	std = tf.broadcast_to(std, x.shape)
	return tf.reshape(std, (N, H, W, C))

class EvoNorm2dS0(tf.keras.layers.Layer):
	def __init__(self, in_channels, groups=32, nonlinear=True):
		super(EvoNorm2dS0, self).__init__()
		self.nonlinear = nonlinear
		self.groups = groups

		def build(self):
			self.gamma = self.add_variable("gamma",
									shape=(1, self.in_channels, 1, 1),
									initializer=tf.initializers.Ones())
			self.beta = self.add_variable("beta",
									shape=(1, self.in_channels, 1, 1),
									initializer=tf.initializers.Zeros())
			if self.nonlinear:
				self.v = self.add_variable("v",
									shape=(1, self.in_channels, 1, 1),
									initializer=tf.initializers.Ones())

		def call(self, x):
			if self.nonlinear:
				num = x * tf.nn.sigmoid(self.v * x)
				return num / group_std(x) * self.gamma + self.beta
			else:
				return x * self.gamma + self.beta

## Mini Inception

In [0]:
# Implementation comes from http://pyimg.co/mac01
def minigooglenet_functional(width, height, depth, classes, norm=EvoNorm2dS0, groups=32):
	def conv_module(x, K, kX, kY, stride, chanDim, padding="same"):
		# define a CONV => EvoNorm pattern
		x = Conv2D(K, (kX, kY), strides=stride, padding=padding)(x)
		
		if isinstance(norm, EvoNorm2dS0):
			layer = norm(in_channels=K, groups=groups)
		else:
			layer = norm(in_channels=K)
		
		x = layer(x)

		# return the block
		return x

	def inception_module(x, numK1x1, numK3x3, chanDim):
		# define two CONV modules, then concatenate across the
		# channel dimension
		conv_1x1 = conv_module(x, numK1x1, 1, 1, (1, 1), chanDim)
		conv_3x3 = conv_module(x, numK3x3, 3, 3, (1, 1), chanDim)
		x = concatenate([conv_1x1, conv_3x3], axis=chanDim)

		# return the block
		return x

	def downsample_module(x, K, chanDim):
		# define the CONV module and POOL, then concatenate
		# across the channel dimensions
		conv_3x3 = conv_module(x, K, 3, 3, (2, 2), chanDim,
			padding="valid")
		pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
		x = concatenate([conv_3x3, pool], axis=chanDim)

		# return the block
		return x

	# initialize the input shape to be "channels last" and the
	# channels dimension itself
	inputShape = (height, width, depth)
	chanDim = -1

	# define the model input and first CONV module
	inputs = Input(shape=inputShape)
	x = conv_module(inputs, 96, 3, 3, (1, 1), chanDim)

	# two Inception modules followed by a downsample module
	x = inception_module(x, 32, 32, chanDim)
	x = inception_module(x, 32, 48, chanDim)
	x = downsample_module(x, 80, chanDim)

	# four Inception modules followed by a downsample module
	x = inception_module(x, 112, 48, chanDim)
	x = inception_module(x, 96, 64, chanDim)
	x = inception_module(x, 80, 80, chanDim)
	x = inception_module(x, 48, 96, chanDim)
	x = downsample_module(x, 96, chanDim)

	# two Inception modules followed by global POOL and dropout
	x = inception_module(x, 176, 160, chanDim)
	x = inception_module(x, 176, 160, chanDim)
	x = AveragePooling2D((7, 7))(x)
	x = Dropout(0.5)(x)

	# softmax classifier
	x = Flatten()(x)
	x = Dense(classes)(x)
	x = Activation("softmax")(x)

	# create the model
	model = Model(inputs, x, name="minigooglenet")

	# return the constructed network architecture
	return model

In [0]:
# One-hot encoding of the labels
y_train_ohe = tf.keras.utils.to_categorical(y_train)
y_test_ohe = tf.keras.utils.to_categorical(y_test)

In [0]:
# Hyperparameters
BATCH_SIZE=128
EPOCHS=60

In [0]:
# Import wandb's Keras callback
from wandb.keras import WandbCallback

In [0]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    "method": "random", #grid, random
    "metric": {
      "name": "accuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "groups": {
            "values": [4, 8, 12, 16, 32]
        },
        "epochs": {
            "values": [10, 20, 30, 40]
        },
        "learning_rate": {
            "values": [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        "optimizer": {
            'values': ["adam", "sgd"]
        }
    }
}

In [19]:
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, entity="sayakpaul", project="evonorm-tf2")

Create sweep with ID: 5aefityj
Sweep URL: https://app.wandb.ai/sayakpaul/evonorm-tf2/sweeps/5aefityj


In [0]:
# Define the training loop
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        "batch_size": 128,
    }
    
    # Initialize a new wandb run
    wandb.init(config=config_defaults, project="evonorm-tf2")
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Call model with the current parameters
    model = minigooglenet_functional(32, 32, 3, 10, norm=EvoNorm2dS0, groups=config.groups)
    
    # Set up optimizer
    if config.optimizer=="sgd":
        optimizer = tf.keras.optimizers.SGD(lr=config.learning_rate, momentum=0.9, decay=1e-2 / config.epochs)
    elif config.optimizer=="adam":
        optimizer = tf.keras.optimizers.Adam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
    
    # Compile model
    model.compile(loss="categorical_crossentropy", optimizer=optimizer,
	    metrics=["accuracy"])
    
    # Train model    
    model.fit(X_train, y_train_ohe,
                    validation_data=(X_test, y_test_ohe),
                    batch_size=config_defaults["batch_size"],
                    epochs=config.epochs,
                    callbacks=[WandbCallback()])

In [27]:
# Initialize sweep
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: s3teel5k with config:
	epochs: 40
	groups: 12
	learning_rate: 1e-05
	optimizer: adam
wandb: Agent Started Run: s3teel5k


Epoch 1/40
391/391 [==============================] - ETA: 0s - loss: 2.0304 - accuracy: 0.2449

wandb: ERROR Can't save model, h5py returned error: Layer EvoNorm2dS0 has arguments in `__init__` and therefore must override `get_config`.


391/391 [==============================] - 16s 42ms/step - loss: 2.0304 - accuracy: 0.2449 - val_loss: 1.8668 - val_accuracy: 0.3264
Epoch 2/40
391/391 [==============================] - 16s 40ms/step - loss: 1.8513 - accuracy: 0.3189 - val_loss: 1.7637 - val_accuracy: 0.3731
Epoch 3/40
391/391 [==============================] - 16s 40ms/step - loss: 1.7483 - accuracy: 0.3608 - val_loss: 1.6546 - val_accuracy: 0.4023
Epoch 4/40
391/391 [==============================] - 16s 40ms/step - loss: 1.6641 - accuracy: 0.3877 - val_loss: 1.5990 - val_accuracy: 0.4254
Epoch 5/40
391/391 [==============================] - 16s 40ms/step - loss: 1.6231 - accuracy: 0.4065 - val_loss: 1.5540 - val_accuracy: 0.4408
Epoch 6/40
391/391 [==============================] - 16s 40ms/step - loss: 1.5895 - accuracy: 0.4231 - val_loss: 1.5325 - val_accuracy: 0.4467
Epoch 7/40
391/391 [==============================] - 16s 40ms/step - loss: 1.5618 - accuracy: 0.4311 - val_loss: 1.5055 - val_accuracy: 0.4567
Epo

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 2.0304 - accuracy: 0.2449

...

wandb: ERROR Can't save model, h5py returned error: Layer EvoNorm2dS0 has arguments in `__init__` and therefore must override `get_config`.


391/391 [==============================] - 16s 41ms/step - loss: 2.0995 - accuracy: 0.2212 - val_loss: 1.9197 - val_accuracy: 0.2955
Epoch 2/30
391/391 [==============================] - 15s 39ms/step - loss: 1.9087 - accuracy: 0.2898 - val_loss: 1.8703 - val_accuracy: 0.3218
Epoch 3/30
391/391 [==============================] - 15s 39ms/step - loss: 1.8495 - accuracy: 0.3197 - val_loss: 1.7777 - val_accuracy: 0.3573
Epoch 4/30
391/391 [==============================] - 15s 39ms/step - loss: 1.7824 - accuracy: 0.3422 - val_loss: 1.7057 - val_accuracy: 0.3816
Epoch 5/30
391/391 [==============================] - 15s 39ms/step - loss: 1.7152 - accuracy: 0.3649 - val_loss: 1.6463 - val_accuracy: 0.3964
Epoch 6/30
391/391 [==============================] - 15s 39ms/step - loss: 1.6750 - accuracy: 0.3832 - val_loss: 1.6132 - val_accuracy: 0.4182
Epoch 7/30
391/391 [==============================] - 15s 38ms/step - loss: 1.6472 - accuracy: 0.3956 - val_loss: 1.5814 - val_accuracy: 0.4312
Epo

Epoch 1/40
391/391 [==============================] - ETA: 0s - loss: 7311061.5000 - accuracy: 0.0990

wandb: ERROR Can't save model, h5py returned error: Layer EvoNorm2dS0 has arguments in `__init__` and therefore must override `get_config`.


391/391 [==============================] - 16s 42ms/step - loss: 7311061.5000 - accuracy: 0.0990 - val_loss: 902938.5625 - val_accuracy: 0.1000
Epoch 2/40
391/391 [==============================] - 15s 40ms/step - loss: 859092.8750 - accuracy: 0.1017 - val_loss: 213325.7188 - val_accuracy: 0.0962
Epoch 3/40
391/391 [==============================] - 15s 40ms/step - loss: 1160460.2500 - accuracy: 0.0995 - val_loss: 267414.2812 - val_accuracy: 0.1000
Epoch 4/40
391/391 [==============================] - 15s 39ms/step - loss: 816655.4375 - accuracy: 0.1039 - val_loss: 1606177.5000 - val_accuracy: 0.1000
Epoch 5/40
391/391 [==============================] - 15s 40ms/step - loss: 894674.0000 - accuracy: 0.1001 - val_loss: 39955.1094 - val_accuracy: 0.0843
Epoch 6/40
391/391 [==============================] - 15s 39ms/step - loss: 700806.9375 - accuracy: 0.1008 - val_loss: 398641.0938 - val_accuracy: 0.1000
Epoch 7/40
391/391 [==============================] - 15s 40ms/step - loss: 1076412.8